In [13]:
# import packages
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
from pandas import DataFrame
import pandas as pd

import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


In [14]:
PATH = 'C:///Users/wchan/OneDrive/Documents/Learning_Coursera/IBM_data_science_professional/BC_data.csv'

bc_df = pd.read_csv(PATH)

In [150]:
filtered_bc_df = bc_df[(bc_df['Neighborhood'].str.contains('Burnaby')) | (bc_df['Neighborhood'].str.contains('Vancouver')) | (bc_df['Neighborhood'].str.contains('Richmond'))| (bc_df['Neighborhood'].str.contains('Surrey'))]
filtered_bc_df[:5]

PostalCode                                       Neighborhood Province  \
0        V5K                 Vancouver (North Hastings-Sunrise)       BC   
1        V5L              Vancouver (North Grandview-Woodlands)       BC   
2        V5M  Vancouver (South Hastings-Sunrise / North Renf...       BC   
3        V5N  Vancouver (South Grandview-Woodlands / NE Kens...       BC   
4        V5P    Vancouver (SE Kensington / Victoria-Fraserview)       BC   

   Latitude  Longitude  
0   49.2807  -123.0397  
1   49.2795  -123.0667  
2   49.2600  -123.0398  
3   49.2551  -123.0667  
4   49.2220  -123.0683

### Explore and cluster the neighborhoods in Vancouver

In [151]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


In [152]:

# create map of New York using latitude and longitude values
map_vancouver = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = filtered_bc_df

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver)  
    
map_vancouver

In [237]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [238]:
CLIENT_ID = '3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH' # your Foursquare ID
CLIENT_SECRET = 'CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH
CLIENT_SECRET:CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z


In [239]:
# CLIENT_ID = 'your-client-ID' # your Foursquare ID
# CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [240]:
neighborhoods.loc[0, 'Neighborhood']

'Vancouver (North Hastings-Sunrise)'

In [241]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood']# neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Vancouver (North Hastings-Sunrise) are 49.2807, -123.0397.


In [242]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH&client_secret=CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z&v=20180605&ll=49.2807,-123.0397&radius=500&limit=100'

In [243]:
results = requests.get(url).json()

In [244]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [245]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name                    categories        lat  \
0        The Fair at the PNE                          Fair  49.282971   
1      Wooden Roller Coaster  Theme Park Ride / Attraction  49.281744   
2            Livestock Barns                          Farm  49.284037   
3                   Playland                    Theme Park  49.281924   
4  Hastings Community Centre                   Event Space  49.280778   

          lng  
0 -123.042109  
1 -123.035128  
2 -123.039278  
3 -123.036258  
4 -123.039176

In [246]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


### Explore Neighborhoods in Vancouver

In [247]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [248]:
yvr_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Vancouver (North Hastings-Sunrise)
Vancouver (North Grandview-Woodlands)
Vancouver (South Hastings-Sunrise / North Renfrew-Collingwood)
Vancouver (South Grandview-Woodlands / NE Kensington)
Vancouver (SE Kensington / Victoria-Fraserview)
Vancouver (South Renfrew-Collingwood)
Vancouver (Killarney)
Vancouver (East Mount Pleasant)
Vancouver (West Kensington / NE Riley Park-Little Mountain)
Vancouver (SE Riley Park-Little Mountain / SW Kensington / NE Oakridge / North Sunset)
Vancouver (SE Oakridge / East Marpole / South Sunset)
Vancouver (West Mount Pleasant / West Riley Park-Little Mountain)
Vancouver (East Fairview / South Cambie)
Vancouver (Strathcona / Chinatown / Downtown Eastside)
Vancouver (NE Downtown / Harbour Centre / Gastown / Yaletown)
Vancouver (Waterfront / Coal Harbour / Canada Place)
Vancouver (South West End)
Vancouver (North West End / Stanley Park)
Vancouver (West Fairview / Granville Island / NE Shaughnessy)
Vancouver (NW Shaughnessy / East Kitsilano / Quilchena)
Vanco

In [249]:
print(yvr_venues.shape)
yvr_venues.head()

(2869, 7)


Neighborhood  Neighborhood Latitude  \
0  Vancouver (North Hastings-Sunrise)                49.2807   
1  Vancouver (North Hastings-Sunrise)                49.2807   
2  Vancouver (North Hastings-Sunrise)                49.2807   
3  Vancouver (North Hastings-Sunrise)                49.2807   
4  Vancouver (North Hastings-Sunrise)                49.2807   

   Neighborhood Longitude                  Venue  Venue Latitude  \
0               -123.0397    The Fair at the PNE       49.282971   
1               -123.0397  Wooden Roller Coaster       49.281744   
2               -123.0397        Livestock Barns       49.284037   
3               -123.0397               Playland       49.281924   
4               -123.0397        Le Petit Saigon       49.281181   

   Venue Longitude                Venue Category  
0      -123.042109                          Fair  
1      -123.035128  Theme Park Ride / Attraction  
2      -123.039278                          Farm  
3      -123.036258                    Theme Park  
4      -123.047309         Vietnamese Restaurant

In [250]:
yvr_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Burnaby (Burnaby Heights / Willingdon Heights /...                     86   
Burnaby (Cascade-Schou / Douglas-Gilpin)                               12   
Burnaby (East Big Bend / Stride Avenue / Edmond...                      9   
Burnaby (Government Road / Lake City / SFU / Bu...                      9   
Burnaby (Lakeview-Mayfield / Richmond Park / Ki...                     22   
Burnaby (Maywood / Marlborough / Oakalla / Wind...                     80   
Burnaby (Parkcrest-Aubrey / Ardingley-Sprott)                          26   
Burnaby (Suncrest / Sussex-Nelson / Clinton-Gle...                     22   
North Vancouver East Central                                            3   
North Vancouver North Central                                           4   
North Vancouver Northwest                                              25   
North Vancouver Northwest Central                                       6   
North Vancouver Outer East                                              1   
North Vancouver South Central                                           5   
North Vancouver Southwest                                              42   
North Vancouver Southwest Central                                      66   
Richmond (Sea Island / YVR)                                            47   
Richmond Central                                                      100   
Richmond North                                                         58   
Richmond Northeast                                                      1   
Richmond South                                                          2   
Richmond Southeast                                                      2   
Richmond Southwest                                                     11   
Richmond West                                                           5   
Surrey                                                                  8   
Surrey East                                                             2   
Surrey Inner Northwest                                                 79   
Surrey Lower West                                                       3   
Surrey North                                                           51   
Surrey Northeast                                                        4   
...                                                                   ...   
Vancouver (East Fairview / South Cambie)                               49   
Vancouver (East Mount Pleasant)                                       100   
Vancouver (Killarney)                                                  20   
Vancouver (NE Downtown / Harbour Centre / Gasto...                    100   
Vancouver (NW Arbutus Ridge)                                           23   
Vancouver (NW Shaughnessy / East Kitsilano / Qu...                    100   
Vancouver (North Grandview-Woodlands)                                 100   
Vancouver (North Hastings-Sunrise)                                     51   
Vancouver (North West End / Stanley Park)                              55   
Vancouver (Pacific Centre)                                            100   
Vancouver (SE Kensington / Victoria-Fraserview)                        23   
Vancouver (SE Kerrisdale / SW Oakridge / West M...                     43   
Vancouver (SE Oakridge / East Marpole / South S...                     26   
Vancouver (SE Riley Park-Little Mountain / SW K...                     41   
Vancouver (SW Downtown)                                               100   
Vancouver (South Grandview-Woodlands / NE Kensi...                     56   
Vancouver (South Hastings-Sunrise / North Renfr...                     37   
Vancouver (South Renfrew-Collingwood)                                  54   
Vancouver (South Shaughnessy / NW Oakridge / NE...                     43   
Vancouver (South West End)                      

In [251]:
print('There are {} uniques categories.'.format(len(yvr_venues['Venue Category'].unique())))

There are 295 uniques categories.


### Analyze Each Neighborhood

In [252]:
# one hot encoding
yvr_onehot = pd.get_dummies(yvr_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
yvr_onehot['Neighborhood'] = yvr_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [yvr_onehot.columns[-1]] + list(yvr_onehot.columns[:-1])
yvr_onehot = yvr_onehot[fixed_columns]

yvr_onehot.head()

Yoga Studio  Accessories Store  Afghan Restaurant  African Restaurant  \
0            0                  0                  0                   0   
1            0                  0                  0                   0   
2            0                  0                  0                   0   
3            0                  0                  0                   0   
4            0                  0                  0                   0   

   Airport  Airport Lounge  Airport Service  Airport Terminal  \
0        0               0                0                 0   
1        0               0                0                 0   
2        0               0                0                 0   
3        0               0                0                 0   
4        0               0                0                 0   

   American Restaurant  Aquarium      ...        \
0                    0         0      ...         
1                    0         0      ...         
2                    0         0      ...         
3                    0         0      ...         
4                    0         0      ...         

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   

   Vietnamese Restaurant  Warehouse Store  Water Park  Waterfront  Wine Bar  \
0                      0                0           0           0         0   
1                      0                0           0           0         0   
2                      0                0           0           0         0   
3                      0                0           0           0         0   
4                      1                0           0           0         0   

   Wine Shop  Women's Store  
0          0              0  
1          0              0  
2          0              0  
3          0              0  
4          0              0  

[5 rows x 295 columns]

In [253]:
yvr_onehot.shape

(2869, 295)

In [254]:
yvr_grouped = yvr_onehot.groupby('Neighborhood').mean().reset_index()
yvr_grouped[:5]

Neighborhood  Yoga Studio  \
0  Burnaby (Burnaby Heights / Willingdon Heights ...     0.000000   
1           Burnaby (Cascade-Schou / Douglas-Gilpin)     0.000000   
2  Burnaby (East Big Bend / Stride Avenue / Edmon...     0.000000   
3  Burnaby (Government Road / Lake City / SFU / B...     0.000000   
4  Burnaby (Lakeview-Mayfield / Richmond Park / K...     0.045455   

   Accessories Store  Afghan Restaurant  African Restaurant  Airport  \
0                0.0                0.0                 0.0      0.0   
1                0.0                0.0                 0.0      0.0   
2                0.0                0.0                 0.0      0.0   
3                0.0                0.0                 0.0      0.0   
4                0.0                0.0                 0.0      0.0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0             0.0              0.0               0.0             0.011628   
1             0.0              0.0               0.0             0.000000   
2             0.0              0.0               0.0             0.000000   
3             0.0              0.0               0.0             0.000000   
4             0.0              0.0               0.0             0.000000   

       ...        Vegetarian / Vegan Restaurant  Video Game Store  \
0      ...                             0.000000          0.011628   
1      ...                             0.083333          0.000000   
2      ...                             0.000000          0.000000   
3      ...                             0.000000          0.000000   
4      ...                             0.000000          0.000000   

   Video Store  Vietnamese Restaurant  Warehouse Store  Water Park  \
0          0.0               0.034884              0.0         0.0   
1          0.0               0.000000              0.0         0.0   
2          0.0               0.000000              0.0         0.0   
3          0.0               0.000000              0.0         0.0   
4          0.0               0.000000              0.0         0.0   

   Waterfront  Wine Bar  Wine Shop  Women's Store  
0         0.0       0.0   0.011628            0.0  
1         0.0       0.0   0.000000            0.0  
2         0.0       0.0   0.000000            0.0  
3         0.0       0.0   0.000000            0.0  
4         0.0       0.0   0.000000            0.0  

[5 rows x 295 columns]

In [255]:
num_top_venues = 5

for hood in yvr_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = yvr_grouped[yvr_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley)----
                  venue  freq
0           Coffee Shop  0.09
1      Sushi Restaurant  0.07
2                  Café  0.05
3  Fast Food Restaurant  0.05
4         Grocery Store  0.03


----Burnaby (Cascade-Schou / Douglas-Gilpin)----
                venue  freq
0            Bus Stop  0.17
1      Sandwich Place  0.17
2    Sushi Restaurant  0.08
3   Food & Drink Shop  0.08
4  Chinese Restaurant  0.08


----Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)----
                venue  freq
0          Restaurant  0.22
1       Deli / Bodega  0.11
2  Italian Restaurant  0.11
3   Indian Restaurant  0.11
4        Burger Joint  0.11


----Burnaby (Government Road / Lake City / SFU / Burnaby Mountain)----
            venue  freq
0    Burger Joint  0.22
1     Golf Course  0.11
2  Baseball Field  0.11
3   Grocery Store  0.11
4     Pizza Place  0.11


----Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway

                   venue  freq
0            Coffee Shop  0.08
1                Brewery  0.05
2  Vietnamese Restaurant  0.04
3       Sushi Restaurant  0.04
4            Pizza Place  0.03


----Vancouver (Killarney)----
                venue  freq
0            Bus Stop  0.15
1                 Bar  0.05
2        Soccer Field  0.05
3  Chinese Restaurant  0.05
4            Pharmacy  0.05


----Vancouver (NE Downtown / Harbour Centre / Gastown / Yaletown)----
         venue  freq
0   Restaurant  0.07
1        Hotel  0.06
2  Coffee Shop  0.05
3   Steakhouse  0.03
4  Pizza Place  0.03


----Vancouver (NW Arbutus Ridge)----
                 venue  freq
0               Bakery  0.13
1  Japanese Restaurant  0.09
2         Dessert Shop  0.04
3         Liquor Store  0.04
4        Moving Target  0.04


----Vancouver (NW Shaughnessy / East Kitsilano / Quilchena)----
                           venue  freq
0                    Coffee Shop  0.10
1         Furniture / Home Store  0.07
2                   

In [256]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [257]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = yvr_grouped['Neighborhood']

for ind in np.arange(yvr_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yvr_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0  Burnaby (Burnaby Heights / Willingdon Heights ...           Coffee Shop   
1           Burnaby (Cascade-Schou / Douglas-Gilpin)              Bus Stop   
2  Burnaby (East Big Bend / Stride Avenue / Edmon...            Restaurant   
3  Burnaby (Government Road / Lake City / SFU / B...          Burger Joint   
4  Burnaby (Lakeview-Mayfield / Richmond Park / K...      Sushi Restaurant   

  2nd Most Common Venue 3rd Most Common Venue          4th Most Common Venue  \
0      Sushi Restaurant                  Café           Fast Food Restaurant   
1        Sandwich Place    Chinese Restaurant  Vegetarian / Vegan Restaurant   
2     Indian Restaurant     Convenience Store                   Burger Joint   
3   Sporting Goods Shop        Baseball Field                  Grocery Store   
4           Yoga Studio     Korean Restaurant                       Bus Stop   

       5th Most Common Venue 6th Most Common Venue  7th Most Common Venue  \
0         Chinese Restaurant              Pharmacy  Vietnamese Restaurant   
1                Snack Place      Sushi Restaurant                   Park   
2                       Park        Sandwich Place          Deli / Bodega   
3         Golf Driving Range           Pizza Place            Golf Course   
4  Middle Eastern Restaurant    Frozen Yogurt Shop         Breakfast Spot   

  8th Most Common Venue       9th Most Common Venue 10th Most Common Venue  
0         Grocery Store                Liquor Store         Sandwich Place  
1           Coffee Shop           Food & Drink Shop           Home Service  
2    Italian Restaurant              Discount Store             Food Truck  
3                  Café  Financial or Legal Service                  Field  
4                  Café        Fast Food Restaurant          Shopping Mall

### Cluster Neighborhoods

In [258]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 8

yvr_grouped_clustering = yvr_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yvr_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 2, 2])

In [259]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

yvr_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
yvr_merged = yvr_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

yvr_merged.head() # check the last columns!

PostalCode                                       Neighborhood Province  \
0        V5K                 Vancouver (North Hastings-Sunrise)       BC   
1        V5L              Vancouver (North Grandview-Woodlands)       BC   
2        V5M  Vancouver (South Hastings-Sunrise / North Renf...       BC   
3        V5N  Vancouver (South Grandview-Woodlands / NE Kens...       BC   
4        V5P    Vancouver (SE Kensington / Victoria-Fraserview)       BC   

   Latitude  Longitude  Cluster Labels         1st Most Common Venue  \
0   49.2807  -123.0397             1.0  Theme Park Ride / Attraction   
1   49.2795  -123.0667             1.0                   Coffee Shop   
2   49.2600  -123.0398             1.0                          Park   
3   49.2551  -123.0667             1.0            Chinese Restaurant   
4   49.2220  -123.0683             1.0                   Pizza Place   

  2nd Most Common Venue  3rd Most Common Venue  4th Most Common Venue  \
0           Coffee Shop                   Park  Vietnamese Restaurant   
1               Brewery                   Café       Sushi Restaurant   
2           Pizza Place            Coffee Shop              Pet Store   
3        Sandwich Place  Vietnamese Restaurant       Sushi Restaurant   
4              Bus Stop      Convenience Store          Grocery Store   

           5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                   Hockey Arena               Stadium            Theme Park   
1                    Pizza Place         Grocery Store    Italian Restaurant   
2          Vietnamese Restaurant         Grocery Store          Liquor Store   
3  Vegetarian / Vegan Restaurant           Pizza Place                  Café   
4                 Sandwich Place       Motorcycle Shop   Fried Chicken Joint   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Event Space  Fast Food Restaurant               Pharmacy  
1      Asian Restaurant     Indian Restaurant                 Bakery  
2        Hardware Store      Department Store         Shop & Service  
3           Coffee Shop         Grocery Store                   Park  
4           Bus Station                  Café       Sushi Restaurant

In [260]:
yvr_merged['Cluster Labels'] = yvr_merged['Cluster Labels'].fillna(0).astype(int)
yvr_merged.head()

PostalCode                                       Neighborhood Province  \
0        V5K                 Vancouver (North Hastings-Sunrise)       BC   
1        V5L              Vancouver (North Grandview-Woodlands)       BC   
2        V5M  Vancouver (South Hastings-Sunrise / North Renf...       BC   
3        V5N  Vancouver (South Grandview-Woodlands / NE Kens...       BC   
4        V5P    Vancouver (SE Kensington / Victoria-Fraserview)       BC   

   Latitude  Longitude  Cluster Labels         1st Most Common Venue  \
0   49.2807  -123.0397               1  Theme Park Ride / Attraction   
1   49.2795  -123.0667               1                   Coffee Shop   
2   49.2600  -123.0398               1                          Park   
3   49.2551  -123.0667               1            Chinese Restaurant   
4   49.2220  -123.0683               1                   Pizza Place   

  2nd Most Common Venue  3rd Most Common Venue  4th Most Common Venue  \
0           Coffee Shop                   Park  Vietnamese Restaurant   
1               Brewery                   Café       Sushi Restaurant   
2           Pizza Place            Coffee Shop              Pet Store   
3        Sandwich Place  Vietnamese Restaurant       Sushi Restaurant   
4              Bus Stop      Convenience Store          Grocery Store   

           5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                   Hockey Arena               Stadium            Theme Park   
1                    Pizza Place         Grocery Store    Italian Restaurant   
2          Vietnamese Restaurant         Grocery Store          Liquor Store   
3  Vegetarian / Vegan Restaurant           Pizza Place                  Café   
4                 Sandwich Place       Motorcycle Shop   Fried Chicken Joint   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0           Event Space  Fast Food Restaurant               Pharmacy  
1      Asian Restaurant     Indian Restaurant                 Bakery  
2        Hardware Store      Department Store         Shop & Service  
3           Coffee Shop         Grocery Store                   Park  
4           Bus Station                  Café       Sushi Restaurant

In [261]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yvr_merged['Latitude'], yvr_merged['Longitude'], yvr_merged['Neighborhood'], yvr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [262]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 0, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
27                 Vancouver (UBC)               0                   NaN   
30      North Vancouver Inner East               0                   NaN   
33   North Vancouver South Central               0        Breakfast Spot   
130                   Surrey South               0                  Café   

          2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
27                          NaN                   NaN                   NaN   
30                          NaN                   NaN                   NaN   
33          Japanese Restaurant                  Café        Sandwich Place   
130  Construction & Landscaping     Food & Drink Shop         Women's Store   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
27                    NaN                   NaN                   NaN   
30                    NaN                   NaN                   NaN   
33                Butcher     Food & Drink Shop            Food Court   
130     Fish & Chips Shop        Farmers Market  Fast Food Restaurant   

    8th Most Common Venue 9th Most Common Venue      10th Most Common Venue  
27                    NaN                   NaN                         NaN  
30                    NaN                   NaN                         NaN  
33                   Food           Flower Shop                 Flea Market  
130                 Field   Filipino Restaurant  Financial or Legal Service

In [263]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 1, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
0                   Vancouver (North Hastings-Sunrise)               1   
1                Vancouver (North Grandview-Woodlands)               1   
2    Vancouver (South Hastings-Sunrise / North Renf...               1   
3    Vancouver (South Grandview-Woodlands / NE Kens...               1   
4      Vancouver (SE Kensington / Victoria-Fraserview)               1   
5                Vancouver (South Renfrew-Collingwood)               1   
6                                Vancouver (Killarney)               1   
7                      Vancouver (East Mount Pleasant)               1   
8    Vancouver (West Kensington / NE Riley Park-Lit...               1   
9    Vancouver (SE Riley Park-Little Mountain / SW ...               1   
10   Vancouver (SE Oakridge / East Marpole / South ...               1   
11   Vancouver (West Mount Pleasant / West Riley Pa...               1   
12            Vancouver (East Fairview / South Cambie)               1   
13   Vancouver (Strathcona / Chinatown / Downtown E...               1   
14   Vancouver (NE Downtown / Harbour Centre / Gast...               1   
15   Vancouver (Waterfront / Coal Harbour / Canada ...               1   
16                          Vancouver (South West End)               1   
17           Vancouver (North West End / Stanley Park)               1   
18   Vancouver (West Fairview / Granville Island / ...               1   
19   Vancouver (NW Shaughnessy / East Kitsilano / Q...               1   
20                       Vancouver (Central Kitsilano)               1   
21                        Vancouver (NW Arbutus Ridge)               1   
22   Vancouver (South Shaughnessy / NW Oakridge / N...               1   
23            Vancouver (Dunbar-Southlands / Musqueam)               1   
24   Vancouver (SE Kerrisdale / SW Oakridge / West ...               1   
25                Vancouver (West Kitsilano / Jericho)               1   
28                             Vancouver (SW Downtown)               1   
34                   North Vancouver Southwest Central               1   
36                           North Vancouver Southwest               1   
39                            West Vancouver Southeast               1   
40                                West Vancouver South               1   
42                          Vancouver (Bentall Centre)               1   
43                          Vancouver (Pacific Centre)               1   
112  Burnaby (East Big Bend / Stride Avenue / Edmon...               1   
113                                       Surrey North               1   
115                             Surrey Inner Northwest               1   
116                             Surrey Outer Northwest               1   
117                                  Surrey Upper West               1   
120                                             Surrey               1   
121                                   Surrey Southwest               1   
138  Burnaby (Government Road / Lake City / SFU / B...               1   
139      Burnaby (Parkcrest-Aubrey / Ardingley-Sprott)               1   
140  Burnaby (Burnaby Heights / Willingdon Heights ...               1   
141  Burnaby (Lakeview-Mayfield / Richmond Park / K...               1   
142           Burnaby (Cascade-Schou / Douglas-Gilpin)               1   
143  Burnaby (Maywood / Marlborough / Oakalla / Win...               1   
144  Burnaby (Suncrest / Sussex-Nelson / Clinton-Gl...               1   
147                                     Richmond North               1   
148                                   Richmond Central               1   
150                        Richmond (Sea Island / YVR)               1   
152                                 Richmond Southwest               1   

            1st Most Common Venue   2nd Most Common Venue  \
0    Theme Park Ride / Attraction             Coffee Shop   
1                     Coffee Shop                 Brewery   
2         

In [264]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 2, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
26   Vancouver (Chaldecutt / South University Endow...               2   
31                        North Vancouver East Central               2   
32                       North Vancouver North Central               2   
35                   North Vancouver Northwest Central               2   
37                           North Vancouver Northwest               2   
38                                West Vancouver North               2   
129                                   Surrey Northeast               2   
151                                      Richmond West               2   

      1st Most Common Venue       2nd Most Common Venue  \
26                    Trail                        Park   
31                    Trail                        Lake   
32                    Trail              Scenic Lookout   
35                    Trail                    Bus Stop   
37                    Trail              Scenic Lookout   
38                 Ski Area                   Ski Lodge   
129  Furniture / Home Store  Construction & Landscaping   
151                   Trail                    Bus Stop   

          3rd Most Common Venue       4th Most Common Venue  \
26                Women's Store  Financial or Legal Service   
31                Women's Store                        Fair   
32                     Mountain               Women's Store   
35   Construction & Landscaping              Scenic Lookout   
37                   Restaurant               Movie Theater   
38                        Trail              Scenic Lookout   
129                       Trail                 Flower Shop   
151              Sandwich Place               Women's Store   

          5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
26           Falafel Restaurant                  Farm        Farmers Market   
31                         Farm        Farmers Market  Fast Food Restaurant   
32           Falafel Restaurant                  Farm        Farmers Market   
35   Financial or Legal Service                  Farm        Farmers Market   
37                   Ski Chalet  Fast Food Restaurant          Skating Rink   
38                  Fabric Shop                  Fair    Falafel Restaurant   
129  Financial or Legal Service                  Farm        Farmers Market   
151                        Farm        Farmers Market  Fast Food Restaurant   

    8th Most Common Venue 9th Most Common Venue      10th Most Common Venue  
26   Fast Food Restaurant                 Field         Filipino Restaurant  
31                  Field   Filipino Restaurant  Financial or Legal Service  
32   Fast Food Restaurant                 Field         Filipino Restaurant  
35   Fast Food Restaurant                 Field         Filipino Restaurant  
37           Dessert Shop   Sporting Goods Shop                   BBQ Joint  
38                   Farm        Farmers Market        Fast Food Restaurant  
129  Fast Food Restaurant                 Field         Filipino Restaurant  
151                 Field   Filipino Restaurant  Financial or Legal Service

In [265]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 3, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
118   Surrey Lower West               3     Convenience Store   
146  Richmond Southeast               3          Home Service   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
118          Home Service           Pizza Place         Women's Store   
146                 Beach           Flea Market        Farmers Market   

          5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
118  Financial or Legal Service    Falafel Restaurant                  Farm   
146        Fast Food Restaurant                 Field   Filipino Restaurant   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
118              Farmers Market  Fast Food Restaurant                  Field  
146  Financial or Legal Service     Fish & Chips Shop          Women's Store

In [266]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 4, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
145  Richmond Northeast               4      Asian Restaurant   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
145         Women's Store     Fish & Chips Shop                  Farm   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
145        Farmers Market  Fast Food Restaurant                 Field   

    8th Most Common Venue       9th Most Common Venue 10th Most Common Venue  
145   Filipino Restaurant  Financial or Legal Service            Flea Market

In [267]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 5, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
29  North Vancouver Outer East               5                  Lake   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
29         Women's Store     Fish & Chips Shop                  Farm   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
29        Farmers Market  Fast Food Restaurant                 Field   

   8th Most Common Venue       9th Most Common Venue 10th Most Common Venue  
29   Filipino Restaurant  Financial or Legal Service            Flea Market

In [268]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 6, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
41   West Vancouver West               6       Harbor / Marina   
149       Richmond South               6       Harbor / Marina   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
41                  Trail      Tapas Restaurant           Fabric Shop   
149                   Bar     Fish & Chips Shop                  Farm   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
41     Falafel Restaurant                  Farm        Farmers Market   
149        Farmers Market  Fast Food Restaurant                 Field   

    8th Most Common Venue       9th Most Common Venue 10th Most Common Venue  
41   Fast Food Restaurant                       Field    Filipino Restaurant  
149   Filipino Restaurant  Financial or Legal Service            Flea Market

In [269]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 7, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
114  Surrey East               7                  Farm        Mattress Store   

    3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
114           Flea Market        Farmers Market  Fast Food Restaurant   

    6th Most Common Venue 7th Most Common Venue       8th Most Common Venue  \
114                 Field   Filipino Restaurant  Financial or Legal Service   

    9th Most Common Venue 10th Most Common Venue  
114     Fish & Chips Shop          Women's Store

In [270]:
yvr_merged.loc[yvr_merged['Cluster Labels'] == 8, yvr_merged.columns[[1] + list(range(5, yvr_merged.shape[1]))]]

Empty DataFrame
Columns: [Neighborhood, Cluster Labels, 1st Most Common Venue, 2nd Most Common Venue, 3rd Most Common Venue, 4th Most Common Venue, 5th Most Common Venue, 6th Most Common Venue, 7th Most Common Venue, 8th Most Common Venue, 9th Most Common Venue, 10th Most Common Venue]
Index: []